In [ ]:
# MIT License
#
#@title Copyright (c) 2021 CCAI Community Authors { display-mode: "form" }
#
# Permission is hereby granted, free of charge, to any person obtaining a
# copy of this software and associated documentation files (the "Software"),
# to deal in the Software without restriction, including without limitation
# the rights to use, copy, modify, merge, publish, distribute, sublicense,
# and/or sell copies of the Software, and to permit persons to whom the
# Software is furnished to do so, subject to the following conditions:
#
# The above copyright notice and this permission notice shall be included in
# all copies or substantial portions of the Software.
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL
# THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING
# FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER
# DEALINGS IN THE SOFTWARE.

## 学習済みモデル（力場）のダウンロード

学習済みのモデルを右記リンクからダウンロードします。[here](https://github.com/Open-Catalyst-Project/ocp/blob/master/MODELS.md)学習済みのモデルはASEのCalculatorクラスとして使用できます。
※今回はリンク先から該当ファイルを直接ダウンロードし、ocpフォルダに格納ください。

In [ ]:
from ocpmodels.common.relaxation.ase_utils import OCPCalculator
import ase.io
from ase.optimize import LBFGS
from ase.build import fcc100, fcc111, add_adsorbate, molecule
import os
from ase.constraints import FixAtoms
import numpy as np
from ase.io import extxyz

import ase
from ase import Atoms, units
from ase.units import Bohr,Rydberg,kJ,kB,fs,Hartree,mol,kcal
from ase.io import read, write
from ase.build import surface, molecule, add_adsorbate
from ase.constraints import FixAtoms, FixedPlane, FixBondLength, ExpCellFilter
from ase.neb import NEB
from ase.vibrations import Vibrations
from ase.thermochemistry import IdealGasThermo
from ase.visualize import view
from ase.build.rotate import minimize_rotation_and_translation
from ase.optimize import BFGS, LBFGS, FIRE
from ase.md import MDLogger
from ase.io import read, write, Trajectory
from ase.build import sort


#モデルの置き場所（ダウンロードしocpフォルダに格納、絶対パスで指定）
checkpoint_path = "C:/Users/****/ocp/gemnet_t_direct_h512_all.pt"
config_yml_path = r"C:/Users/****/ocp/configs/s2ef/all/gemnet/gemnet-dT.yml"

#Define the calculator（CPUの場合、cpu = True）
calc = OCPCalculator(config_yml=config_yml_path, checkpoint=checkpoint_path, cpu = False)

## COの吸着エネルギー計算
典型的な例として、FCC(111)面のトップサイトにおけるCOの吸着エネルギーを各種金属に対して計算します。

In [ ]:
#COと金属の吸着構造最適化とエネルギーの計算
adslab = fcc111("Cu", size=(3, 3, 3))
adsorbate = molecule("CO")
add_adsorbate(adslab, adsorbate, 3, offset=(1, 1)) # adslab = adsorbate + slab

# tag all slab atoms below surface as 0, surface as 1, adsorbate as 2
tags = np.zeros(len(adslab))
tags[18:27] = 1
tags[27:] = 2
adslab.set_tags(tags)

# Fixed atoms are prevented from moving during a structure relaxation. 
# We fix all slab atoms beneath the surface. 
cons= FixAtoms(indices=[atom.index for atom in adslab if (atom.tag == 0)])
adslab.set_constraint(cons)
adslab.center(vacuum=13.0, axis=2)
adslab.set_pbc(False)

# Set up the calculator
adslab.set_calculator(calc)
os.makedirs('data', exist_ok=True)

# Define structure optimizer - LBFGS. Run for 100 steps, 
# or if the max force on all atoms (fmax) is below 0 ev/A.
# fmax is typically set to 0.01-0.05 eV/A, 
# for this demo however we run for the full 100 steps.
dyn = LBFGS(adslab, trajectory="data/" + "_co_relax.traj")
dyn.run(fmax=0.05, steps=100)
traj = ase.io.read("data/" + "_co_relax.traj", ":")
# convert traj format to extxyz format (used by OC20 dataset)
columns = (['symbols','positions', 'move_mask', 'tags'])
with open("data/" + "_co_relax.extxyz",'w') as f:
    extxyz.write_xyz(f, traj, columns=columns)



In [ ]:
#金属のみの構造最適化とエネルギー計算
adslab = fcc111("Cu", size=(3, 3, 3))
tags = np.zeros(len(adslab))
tags[18:27] = 1
tags[27:] = 2
adslab.set_tags(tags)
# Fixed atoms are prevented from moving during a structure relaxation. 
# We fix all slab atoms beneath the surface. 
cons= FixAtoms(indices=[atom.index for atom in adslab if (atom.tag == 0)])
adslab.set_constraint(cons)
adslab.center(vacuum=13.0, axis=2)
adslab.set_pbc(False)
# Set up the calculator
adslab.set_calculator(calc)
os.makedirs('data', exist_ok=True)
# Define structure optimizer - LBFGS. Run for 100 steps, 
# or if the max force on all atoms (fmax) is below 0 ev/A.
# fmax is typically set to 0.01-0.05 eV/A, 
# for this demo however we run for the full 100 steps.
dyn = LBFGS(adslab, trajectory="data/" + "_relax.traj")
dyn.run(fmax=0.05, steps=100)
traj_metal = ase.io.read("data/" + "_relax.traj", ":")
# convert traj format to extxyz format (used by OC20 dataset)
columns = (['symbols','positions', 'move_mask', 'tags'])
with open("data/" + "_relax.extxyz",'w') as f:
    extxyz.write_xyz(f, traj_metal, columns=columns)

#吸着エネルギーの計算
#最適化後の吸着構造をロードし、エネルギーを出力
final_structure = traj[-1]
relaxed_energy = final_structure.get_potential_energy()
print(f'Relaxed absolute energy = {relaxed_energy} eV')

#最適化後の金属構造をロードし、エネルギーを出力
final_structure = traj_metal[-1]
raw_slab_energy = final_structure.get_potential_energy()
print(f'Raw slab energy = {raw_slab_energy} eV')


最後に吸着エネルギーと構造を表示します。

In [ ]:
import matplotlib.pyplot as plt
from ase.visualize.plot import plot_atoms

adsorbate = Atoms("CO").get_chemical_symbols()
# For clarity, we define arbitrary gas reference energies here.
# A more detailed discussion of these calculations can be found in the corresponding paper's SI. 
gas_reference_energies = {'H': .3, 'O': .45, 'C': .35, 'N': .50}
adsorbate_reference_energy = 0
for ads in adsorbate:
    adsorbate_reference_energy += gas_reference_energies[ads]
print(f'Adsorbate reference energy = {adsorbate_reference_energy} eV\n')
adsorption_energy = relaxed_energy - raw_slab_energy - adsorbate_reference_energy
print(f'Adsorption energy: {adsorption_energy} eV')

#初期構造の表示
fig, ax = plt.subplots(1, 1)
ase.visualize.plot.plot_atoms(traj[len(traj) - 1], 
                              ax, 
                              radii=0.8, 
                              rotation=("-75x, 45y, 10z"))
plt.show()
